In [1]:
%matplotlib inline
import json
import os            
import pandas as pd    
import numpy as np

The first thing we should do is analysing our dataframe concerning calls to 112. To simplify this operation, we already subsetted the initial dataframe by taking into account only the data concerning Den Haag.  
Let's then analyse this last dataframe that we called Den_Haag_calls. 

In [2]:
path='Calls data/Den_Haag_calls.csv' #defining relative path to our csv file
DH_calls=pd.read_csv(path)
DH_calls.head() #exploring the first five rows

C:\ProgramData\Anaconda3\envs\gds\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,pmeId,pmeTimeStamp,pmeProtocol1,pmeProtocol2,pmeTarget,pmeMessage,pmePrio,pmePrioLevel,pmeDienst,pmeStrippedMessage,...,pme_strId,pme_wplId,pme_gemId,pme_proId,pme_vrgId,pmeCapCodes,pmeLifeLiner,pme_catId,wplNam,gemName
0,12284702,2017-01-01 00:00:37,FLEX-A,1600,AORG,A1 Goudsbloemlaan 71-79 DHG 2565CP : 15101 Rit...,A1,1,A,15101 Ritnummer: 1,...,199928,4896,391,12,5170,1520001,\N,\N,Den Haag,Den Haag
1,12284730,2017-01-01 00:04:52,FLEX-A,1600,AORG,P 1 Buitenbrand afval/rommel Drebbelstraat DHG...,P 1,1,B,Buitenbrand afval/rommel 7630,...,199712,4896,391,12,5170,"1500148,1500632,1503902,2029568",\N,\N,Den Haag,Den Haag
2,12284734,2017-01-01 00:05:29,FLEX-A,1600,AORG,P 1 Gebouwbrand woning (VK: 12) Berberisstraat...,P 1,1,B,Gebouwbrand woning (VK: 12) 9591 7830 9634 7850,...,199326,4896,391,12,5170,"1500050,1500142,1500624,1500634,1503902,2029568",\N,\N,Den Haag,Den Haag
3,12284777,2017-01-01 00:11:56,FLEX-A,1600,AORG,P 1 OMS Geverifieerd (brandmelding OMS) De Wit...,P 1,1,B,Melding Via Openbaar Meldsysteem Geverifieerd ...,...,201019,4896,391,12,5170,"1500142,1503902,2029568",\N,\N,Den Haag,Den Haag
4,12284781,2017-01-01 00:12:10,FLEX-A,1600,AORG,A1 Zuiderparklaan 81-89 DHG 2574HC : 15129 Rit...,A1,1,A,15129 Ritnummer: 7,...,202018,4896,391,12,5170,1520029,\N,\N,Den Haag,Den Haag


Let's also explore the shape of the dataframe. 

Knowing that the last column of this dataframe only contains the name of the chosen municipality we can simply drop it (knowing that it contains the same information for all our rows). 

In [3]:
DH_calls.drop('gemName',axis=1, inplace=True)
DH_calls.shape #checking if our operation was successful 

(256984, 30)

For our analysis we'll only use data about ambulance calls in Den Haag. To do that, we have to subset our dataframe on the basis of the column named as 'pmeDienst'. **(A=Ambulance, B=Brandweer, P=Politie)**

In [4]:
DH_amb_calls=DH_calls.loc[DH_calls['pmeDienst']=='A'].reset_index(drop=True) #creating new dataframe that only contains ambulance calls 
DH_amb_calls.shape #checking if our operation was successful 

(212566, 30)

Now our previous column of interest ('pmeDienst') contains only one value (A). Thus, we can drop it from our new dataframe. 

In [5]:
DH_amb_calls.drop('pmeDienst', axis=1, inplace=True)
DH_amb_calls.shape #checking if our operation was successful 

(212566, 29)

It's really important to realise that our dataframe is still 'dirty'. For istance, the second column ('pmeTimeStamp') contains four different variables (year, day, month, hour). What we must do in the first place is to separate these four variables into diffent columns of interest.  
Only when we'll achieve that, we can create 4 different dataframes based on the year of the calls. (i.e. dataframe 1 <--> 2017; dataframe 2 <--> 2018 etc.).

Let's first analyse the type of our column data. 

In [6]:
DH_amb_calls['pmeTimeStamp'].dtypes

dtype('O')

Even though this column contains numbers (year, month, day, time), its data is classified as string objects. 

We can convert the 'pmeTimeStamp' column values to datetime type. 

In [7]:
DH_amb_calls['pmeTimeStamp']=pd.to_datetime(DH_amb_calls['pmeTimeStamp'])

We can now create four separate columns containing the year, month, day and time of the call.

In [8]:
#year column
DH_amb_calls['Year']=DH_amb_calls['pmeTimeStamp'].dt.year
#month column
DH_amb_calls['Month']=DH_amb_calls['pmeTimeStamp'].dt.month
#day column
DH_amb_calls['Day']=DH_amb_calls['pmeTimeStamp'].dt.day
#time column
DH_amb_calls['Time']=DH_amb_calls['pmeTimeStamp'].dt.time

In [9]:
DH_amb_calls.head()

,pmeId,pmeTimeStamp,pmeProtocol1,pmeProtocol2,pmeTarget,pmeMessage,pmePrio,pmePrioLevel,pmeStrippedMessage,pmeStraat,...,pme_proId,pme_vrgId,pmeCapCodes,pmeLifeLiner,pme_catId,wplNam,Year,Month,Day,Time
0,12284702,2017-01-01 00:00:37,FLEX-A,1600,AORG,A1 Goudsbloemlaan 71-79 DHG 2565CP : 15101 Rit...,A1,1,15101 Ritnummer: 1,Goudsbloemlaan,...,12,5170,1520001,\N,\N,Den Haag,2017,1,1,00:00:37
1,12284781,2017-01-01 00:12:10,FLEX-A,1600,AORG,A1 Zuiderparklaan 81-89 DHG 2574HC : 15129 Rit...,A1,1,15129 Ritnummer: 7,Zuiderparklaan,...,12,5170,1520029,\N,\N,Den Haag,2017,1,1,00:12:10
2,12284809,2017-01-01 00:15:54,FLEX-A,1600,AORG,A1 Middenstip 1-9 DHG 2492VT : 15142 Ritnr: 1-9,A1,1,15142 Ritnummer: 1-9,Middenstip,...,12,5170,1520042,\N,\N,Den Haag,2017,1,1,00:15:54
3,12284833,2017-01-01 00:17:52,FLEX-A,1600,AORG,A1 Keukenhoflaan 11-19 DHG 2548PC : 15115 Ritn...,A1,1,15115 Ritnummer: 11,Keukenhoflaan,...,12,5170,1520015,\N,\N,Den Haag,2017,1,1,00:17:52
4,12284971,2017-01-01 00:30:41,FLEX-A,1600,AORG,A2 Ambulancepost Mangaan Mangaanstraat 80-88 D...,A2,2,Ambulancepost Mangaan Opstellen bij ambulancep...,Mangaanstraat,...,12,5170,1520026,\N,\N,Den Haag,2017,1,1,00:30:41


Now that we created this new dataframe, containing ambulance calls' year, month and day, we can drop the initial column that contained "time information" about the calls. 

In [10]:
DH_amb_calls= DH_amb_calls.drop('pmeTimeStamp',axis=1)
DH_amb_calls.shape #checking if our operation was successful 

(212566, 32)

From this dataset we can exclude all the columns that do not contain useful information for our analysis.  
In particular, we are only going to take into account the following columns: 
* Year 
* Month 
* Day 
* Time
* pmeLatitude = wgs84
* pmeLongitude = wgs84

In [11]:
chosen_columns= ['Year','Month','Day','Time','pmeLongitude','pmeLatitude']
DH_amb_calls_new= DH_amb_calls[chosen_columns]
DH_amb_calls_new.shape #checking if our operation was successful 

(212566, 6)

In [12]:
# Time series plots
# Maps

In [13]:
DH_amb_calls_new.head()

,Year,Month,Day,Time,pmeLongitude,pmeLatitude
0,2017,1,1,00:00:37,4.2607498,52.0768602
1,2017,1,1,00:12:10,4.2771779,52.0673801
2,2017,1,1,00:15:54,4.4047482,52.0735557
3,2017,1,1,00:17:52,4.2963787,52.0300043
4,2017,1,1,00:30:41,4.2515080,52.0495713


In [14]:
DH_amb_calls_new.tail()

,Year,Month,Day,Time,pmeLongitude,pmeLatitude
212561,2020,8,30,19:05:44,4.291111,52.069858
212562,2020,8,30,19:08:55,4.252024,52.049683
212563,2020,8,30,19:09:27,4.34559,52.083516
212564,2020,8,30,19:23:37,4.34559,52.083516
212565,2020,8,30,19:28:40,4.272161,52.106352


In [15]:
DH_amb_calls_new=DH_amb_calls_new.rename(columns={'pmeLongitude':'Longitude','pmeLatitude':'Latitude'})
DH_amb_calls_new.head()

,Year,Month,Day,Time,Longitude,Latitude
0,2017,1,1,00:00:37,4.2607498,52.0768602
1,2017,1,1,00:12:10,4.2771779,52.0673801
2,2017,1,1,00:15:54,4.4047482,52.0735557
3,2017,1,1,00:17:52,4.2963787,52.0300043
4,2017,1,1,00:30:41,4.2515080,52.0495713


Let's also check if inside the 'Latitude' and 'Longitude' columns we find values that are not numerical. 

In [16]:
len(DH_amb_calls_new[DH_amb_calls_new['Latitude']==r'\N'])

9

We can drop these rows beacuse they do not contains geographic data about ambulance calls. (also they are only 9 lines!)

In [17]:
DH_amb_calls_new=DH_amb_calls_new.drop(DH_amb_calls_new[DH_amb_calls_new['Latitude']==r'\N'].index, axis=0).reset_index(drop=True)
DH_amb_calls_new.shape

(212557, 6)

In [18]:
len(DH_amb_calls_new[DH_amb_calls_new['Longitude']==r'\N'])

0

What we can do now is to check for possible duplicates in our dataframe. In other words, if two ambulance calls have the same coordinates and the same date (year, month, day, time) we can simply imply that those calls are duplicates, thus, we are going to keep only one call out of those. 

In [19]:
duplicates_index=DH_amb_calls_new[DH_amb_calls_new.duplicated(keep='first')].index
DH_amb_calls_new=DH_amb_calls_new.drop(duplicates_index, axis=0).reset_index(drop=True)
DH_amb_calls_new.shape

(212515, 6)

## Last part of EDA on ambulance calls dataset

As we previously described, from this new dataset we can create four separate datasets depending on the year of the call.  
* Dataframe 1: Ambulance calls , 2017
* Dataframe 2: Ambulance calls , 2018
* Dataframe 3: Ambulance calls , 2019
* Dataframe 4: Ambulance calls , 2020  

Let's then subset these new dataframes from our DH_amb_calls_new dataframe. 

In [20]:
#2017 ambulance calls dataframe 
DH_amb_2017=DH_amb_calls_new.loc[DH_amb_calls_new['Year']== 2017].reset_index(drop=True)
#2018 ambulance calls dataframe 
DH_amb_2018=DH_amb_calls_new.loc[DH_amb_calls_new['Year']== 2018].reset_index(drop=True)
#2019 ambulance calls dataframe 
DH_amb_2019=DH_amb_calls_new.loc[DH_amb_calls_new['Year']== 2019].reset_index(drop=True)
#2020 ambulance calls dataframe 
DH_amb_2020=DH_amb_calls_new.loc[DH_amb_calls_new['Year']== 2020].reset_index(drop=True)

Let's check the shape of each one of these dataframes to have a first understanding of the 'quantity' of data available each year. 

In [21]:
print('2017 dataset rows:',DH_amb_2017.shape[0])
print('2018 dataset rows:',DH_amb_2018.shape[0])
print('2019 dataset rows:',DH_amb_2019.shape[0])
print('2020 dataset rows:',DH_amb_2020.shape[0])

2017 dataset rows: 60233
2018 dataset rows: 58729
2019 dataset rows: 57339
2020 dataset rows: 36214


Let's then drop the Year column for these specific dataframes. 

In [22]:
#2017 
DH_amb_2017=DH_amb_2017.drop('Year',axis=1)
#2018 
DH_amb_2018=DH_amb_2018.drop('Year',axis=1)
#2019 
DH_amb_2019=DH_amb_2019.drop('Year',axis=1)
#2020 
DH_amb_2020=DH_amb_2020.drop('Year',axis=1)

Taking into account both our hypothesis for the future regression model that we are going to use and the limited dimensions of the 2020 dataframe, we will exclude this last dataframe from our analysis.  

Thus, let us save the datasets for 2017, 2018, 2019 in three csv files that we are going to clean in the future. 

In [23]:
#saving 2017 dataframe 
DH_amb_2017.to_csv('Calls data/Calls_2017.csv',index=False)
#saving 2018 dataframe 
DH_amb_2018.to_csv('Calls data/Calls_2018.csv',index=False)
#saving 2019 dataframe
DH_amb_2019.to_csv('Calls data/Calls_2019.csv',index=False)

## Operational summary for our final three datasets 

Our last three dataframe contain 6 columns: 
* Year of the call 
* Month of the call 
* Day of the call
* Time of the call 
* Longitude of the call 
* Latitude of the call